In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Load the dataset
hotels_df = pd.read_csv('https://raw.githubusercontent.com/kristxna/Datasets/refs/heads/main/hotels.csv')

In [3]:
hotels_df.columns

Index(['Room_ID', 'Floor', 'Square_Feet', 'Cleaning_Time_Hours'], dtype='object')

In [4]:
hotels_df.head()

,Room_ID,Floor,Square_Feet,Cleaning_Time_Hours
0,1,3,682,1.814545
1,2,9,223,0.562727
2,3,7,506,1.334545
3,4,1,561,1.484545
4,5,14,424,1.110909


In [5]:
print("\n=================== Original Optimization Model ===================")

# Extract relevant data
floors = hotels_df['Floor'].unique()
rooms = hotels_df['Room_ID'].unique()
num_attendants = 8
max_sqft_per_day = 3500
max_overtime_hours = 2
base_wage = 25
extra_floor_cost = 75

def get_room_data():
    room_data = {}
    for _, row in hotels_df.iterrows():
        floor, room, sqft, time = row['Floor'], row['Room_ID'], row['Square_Feet'], row['Cleaning_Time_Hours']
        if floor not in room_data:
            room_data[floor] = []
        room_data[floor].append((room, sqft, time))
    return room_data

room_data = get_room_data()

# Initialize model 
model = gp.Model("Hotel Cleaning Scheduling")

# Decision Variables
x = model.addVars(num_attendants, floors, rooms, vtype=GRB.BINARY, name="Assign")
f = model.addVars(num_attendants, floors, vtype=GRB.BINARY, name="FloorAssign")
overtime = model.addVars(num_attendants, vtype=GRB.INTEGER, lb=0, ub=max_overtime_hours, name="Overtime")
over_sqft = model.addVars(num_attendants, vtype=GRB.BINARY, name="OverSqft")
floor_violation = model.addVars(num_attendants, vtype=GRB.INTEGER, lb=0, name="FloorViolation")

# Objective Function: Minimize cost
model.setObjective(
    gp.quicksum(base_wage * 8 + 1.5 * base_wage * overtime[i] + extra_floor_cost * floor_violation[i] 
                + 2 * base_wage * over_sqft[i] for i in range(num_attendants)), GRB.MINIMIZE)

# Constraints
# Ensure all rooms are assigned to one attendant
for k in floors:
    for j in room_data[k]:
        model.addConstr(gp.quicksum(x[i, k, j[0]] for i in range(num_attendants)) == 1, f"RoomAssignment_{k}_{j[0]}")

# Floor assignment consistency
for i in range(num_attendants):
    for k in floors:
        for j in room_data[k]:
            model.addConstr(x[i, k, j[0]] <= f[i, k], f"FloorConsistency_{i}_{k}_{j[0]}")

# Each attendant cleans between 2 to 4 floors
for i in range(num_attendants):
    model.addConstr(gp.quicksum(f[i, k] for k in floors) >= 2, f"MinFloors_{i}")
    model.addConstr(gp.quicksum(f[i, k] for k in floors) <= 4, f"MaxFloors_{i}")

# Track extra floor violations (if more than 2 floors, record the extra count)
for i in range(num_attendants):
    model.addConstr(floor_violation[i] >= gp.quicksum(f[i, k] for k in floors) - 2, f"TrackFloorViolations_{i}")

# Ensure attendants do not exceed 3500 sqft before extra pay applies
for i in range(num_attendants):
    model.addConstr(
        gp.quicksum(x[i, k, j[0]] * j[1] for k in floors for j in room_data[k]) <= max_sqft_per_day + over_sqft[i] * 5000,
        f"SqFtLimit_{i}"
    )

# Ensure attendants work within 8 hours with a maximum of 2 overtime hours
for i in range(num_attendants):
    model.addConstr(gp.quicksum(x[i, k, j[0]] * j[2] for k in floors for j in room_data[k]) <= 8 + overtime[i], f"WorkHours_{i}")

# Solve the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    total_overtime = sum(overtime[i].x for i in range(num_attendants))
    total_floor_violations = sum(floor_violation[i].x for i in range(num_attendants))
    print(f"Optimal Staffing Cost for the Day: ${model.objVal:.2f}")
    print(f"Total Overtime Hours: {total_overtime}")
    print(f"Total Floor Violations: {total_floor_violations}")

    print("\n🔹 Attendant Summary:")
    for i in range(num_attendants):
        assigned_floors = [k for k in floors if f[i, k].x > 0.5]
        num_floors = len(assigned_floors)
        total_sqft_cleaned = sum(x[i, k, j[0]].x * j[1] for k in floors for j in room_data[k])
        overtime_hours = int(overtime[i].x)
        floor_violations = int(floor_violation[i].x)
        
        extra_sqft_pay = 2 * base_wage * (total_sqft_cleaned > max_sqft_per_day)
        extra_floor_pay = extra_floor_cost * floor_violations
        
        total_wage = (base_wage * 8) + (1.5 * base_wage * overtime_hours) + extra_sqft_pay + extra_floor_pay

        print(f"Attendant {i+1}: Floors [{', '.join(map(str, assigned_floors))}], "
              f"Overtime: {overtime_hours} hours, Total Sqft Cleaned: {total_sqft_cleaned:.2f}, "
              f"Floor Violations: {floor_violations}, Total Wage: ${total_wage:.2f}")
else:
    print("No optimal solution found.")


=================== Original Optimization Model ===================
Set parameter Username
Set parameter LicenseID to value 2609994
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 508 rows, 5960 columns and 2440 nonzeros
Model fingerprint: 0x4b78e7b7
Variable types: 0 continuous, 5960 integer (5944 binary)
Coefficient statistics:
  Matrix range     [6e-01, 5e+03]
  Objective range  [4e+01, 8e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 3162.5000000
Presolve removed 16 rows and 5408 columns
Presolve time: 0.01s
Presolved: 492 rows, 552 columns, 2216 nonzeros
Variable types: 0 continuous, 552 integer (536 binary)

Root relaxation: objective 1.660511e+03, 594 iterations, 0.

In [6]:
# Verify all rooms are assigned
all_rooms_cleaned = True
uncleaned_rooms = []

for k in floors:
    for j in room_data[k]:
        assigned = sum(x[i, k, j[0]].x for i in range(num_attendants))  # Summing assignments
        if assigned != 1:
            all_rooms_cleaned = False
            uncleaned_rooms.append((j[0], k))  # Collect uncleaned rooms

if all_rooms_cleaned:
    print("All rooms were assigned and cleaned.")
else:
    print("Some rooms were not cleaned!")
    print(f"Uncleaned rooms: {uncleaned_rooms}")

All rooms were assigned and cleaned.


In [7]:
# (f) Model relaxation
print("\n=================== Relaxed Model ===================")
relaxed_model = model.relax() 
relaxed_model.optimize()

if relaxed_model.status == GRB.OPTIMAL:
    print(f"(f) Relaxed Model Cost: ${relaxed_model.objVal:.2f}")



=================== Relaxed Model ===================
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 508 rows, 5960 columns and 2440 nonzeros
Model fingerprint: 0x41ad9c95
Coefficient statistics:
  Matrix range     [6e-01, 5e+03]
  Objective range  [4e+01, 8e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 5408 columns
Presolve time: 0.01s
Presolved: 500 rows, 560 columns, 2336 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6605114e+03   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       8 DPushes remaining with DInf 0.0000000e+00                 0s

     468 PPushes remaining with PInf 0.0000000e+00                 0s
       0 PP

In [8]:
# Part (g) Convert binary to continuous and solve manually relaxed problem
print("\n=================== Manually Relaxed Model ===================")

# Make a copy of the model to manually relax it
manual_relax_model = model.copy()

# Convert all decision variables to continuous
for v in manual_relax_model.getVars():
    v.vtype = GRB.CONTINUOUS

manual_relax_model.optimize()

if manual_relax_model.status == GRB.OPTIMAL:
    print(f"(g) Manually Relaxed Solution Cost: ${manual_relax_model.objVal:.2f}")


=================== Manually Relaxed Model ===================
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 508 rows, 5960 columns and 2440 nonzeros
Model fingerprint: 0x41ad9c95
Coefficient statistics:
  Matrix range     [6e-01, 5e+03]
  Objective range  [4e+01, 8e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 5408 columns
Presolve time: 0.01s
Presolved: 500 rows, 560 columns, 2336 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6605114e+03   0.000000e+00   0.000000e+00      0s

Use crossover to convert LP symmetric solution to basic solution...
Crossover log...

       8 DPushes remaining with DInf 0.0000000e+00                 0s

     468 PPushes remaining with PInf 0.0000000e+00                 0s
  

In [9]:
print(f"Binary Model Cost: ${model.objVal:.2f}")
print(f"Relaxed Model Cost: ${relaxed_model.objVal:.2f}")

Binary Model Cost: $1750.00
Relaxed Model Cost: $1660.51


In [10]:
print(f"Before 2x Overtime: Cost = ${model.objVal:.2f}, Overtime Hours = {total_overtime}, Floor Violations = {total_floor_violations}")

Before 2x Overtime: Cost = $1750.00, Overtime Hours = 4.0, Floor Violations = 0.0


In [11]:
# Part (i) Adjust overtime multiplier to 2.0 and re-optimize
print("\n=================== Updated Model with 2x Overtime Pay ===================")

# Restore decision variables to binary (in case they were changed)
for v in model.getVars():
    if "Assign" in v.varName or "FloorAssign" in v.varName:
        v.vtype = GRB.BINARY

overtime_multiplier = 2.0

# Update Objective Function
model.setObjective(
    gp.quicksum(base_wage * 8 + base_wage * overtime_multiplier * overtime[i] for i in range(num_attendants)) +
    gp.quicksum(extra_floor_cost * floor_violation[i] for i in range(num_attendants)) +
    gp.quicksum(2 * base_wage * over_sqft[i] for i in range(num_attendants)),
    GRB.MINIMIZE
)

model.optimize()

# Display Results
if model.status == GRB.OPTIMAL:
    total_overtime = sum(overtime[i].x for i in range(num_attendants))
    total_floor_violations = sum(floor_violation[i].x for i in range(num_attendants))
    
    print(f"(i) Updated Optimal Cost with 2x Overtime: ${model.objVal:.2f}")
    print(f"(i) Total Overtime Hours: {total_overtime}")
    print(f"(i) Total Floor Violations: {total_floor_violations}")
else:
    print("No optimal solution found.")



=================== Updated Model with 2x Overtime Pay ===================
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 24.1.0 24B83)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 508 rows, 5960 columns and 2440 nonzeros
Model fingerprint: 0x47e725bc
Variable types: 0 continuous, 5960 integer (5944 binary)
Coefficient statistics:
  Matrix range     [6e-01, 5e+03]
  Objective range  [5e+01, 8e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 4e+03]

Loaded MIP start from previous solve with objective 1800

Presolve removed 16 rows and 5408 columns
Presolve time: 0.01s
Presolved: 492 rows, 552 columns, 2216 nonzeros
Variable types: 0 continuous, 552 integer (536 binary)

Root relaxation: objective 1.680682e+03, 594 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl

In [12]:
print(f"After 2x Overtime: Cost = ${model.objVal:.2f}")

After 2x Overtime: Cost = $1775.00
